In [1]:
# import autosklearn.regression
import sklearn.model_selection
import sklearn.metrics
import os
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

<h2>02 Data Collection</h2>

In [2]:
df = []
src = r"C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\train\train"
for file in os.listdir(src):
    df.append(pd.read_csv(os.path.join(src,file)))
for i in df:
    print(list(i))
df = pd.concat(df)
df = df.fillna(0)
df[-10:]

['index', 'time', '1Air Humidity', '1AirTemperature', '1LightHigh', '1LightLow', '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed', '13WindDirection', '23SoilWC ค.ชื้นดิน', '23SoilTemp.อุณหภูมิดิน', '23SoilEC', '23SoilpH', '26N', '26P', '26K', '29SoilTension', 'flow1']
['index', 'time', '1Air Humidity', '1AirTemperature', '1LightHigh', '1LightLow', '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed', '13WindDirection', '23SoilTemp.อุณหภูมิดิน', '23SoilEC', '23SoilpH', '26N', '26P', '26K', '29SoilTension', 'flow1']
['index', 'time', '1Air Humidity', '1AirTemperature', '1LightHigh', '1LightLow', '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed', '13WindDirection', '23SoilWC ค.ชื้นดิน', '23SoilTemp.อุณหภูมิดิน', '23SoilEC', '23SoilpH', '26N', '26P', '26K', '29SoilTension', 'flow1']
['index', 'time', '1Air Humidity', '1AirTemperature', '1LightHigh', '1LightLow', '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed', '13WindDirection', '23SoilWC ค.ชื้นดิน', '23SoilTemp.อุณ

,index,time,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,13WindDirection,23SoilWC ค.ชื้นดิน,23SoilTemp.อุณหภูมิดิน,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1
104,12_105,04:40:16.090733+00,44.1,31.2,0.0,47.64,0.0,0.0,0.5,3.0,38.8,22.8,0.11,5.0,21.0,95.0,87.0,105.0,0.0
105,12_106,04:42:16.132055+00,44.2,31.2,0.0,49.50,0.0,0.0,0.1,1.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,111.0,0.0
106,12_107,04:44:16.107327+00,45.2,30.8,0.0,50.03,0.0,0.0,1.2,7.0,37.4,22.8,0.11,5.0,21.0,95.0,87.0,113.0,0.0
107,12_108,04:46:16.11276+00,45.9,32.0,0.0,50.06,0.0,0.0,0.6,1.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,110.0,0.0
108,12_109,04:48:16.120498+00,41.4,33.1,0.0,50.36,0.0,0.0,0.7,7.0,37.6,22.8,0.11,5.0,21.0,95.0,87.0,108.0,0.0
109,12_110,04:50:16.123489+00,43.0,33.0,0.0,52.30,0.0,0.0,1.0,5.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,109.0,0.0
110,12_111,04:52:18.185763+00,42.5,32.5,0.0,53.76,0.0,0.0,0.8,4.0,37.4,22.8,0.11,5.0,21.0,95.0,87.0,0.0,0.0
111,12_112,04:54:16.141528+00,41.7,33.1,0.0,54.86,0.0,0.0,0.8,4.0,37.4,22.8,0.11,5.1,21.0,95.0,87.0,109.0,0.0
112,12_113,04:56:20.078654+00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,2.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,104.0,0.0
113,12_114,04:58:16.116687+00,44.8,32.5,0.0,58.81,0.0,0.0,1.2,7.0,37.4,22.8,0.11,5.1,21.0,95.0,87.0,113.0,0.0


In [3]:
df.shape

(1340, 19)

In [4]:
df.columns

Index(['index', 'time', '1Air Humidity', '1AirTemperature', '1LightHigh',
       '1LightLow', '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed',
       '13WindDirection', '23SoilWC ค.ชื้นดิน', '23SoilTemp.อุณหภูมิดิน',
       '23SoilEC', '23SoilpH', '26N', '26P', '26K', '29SoilTension', 'flow1'],
      dtype='object')

In [5]:
from datetime import datetime


In [6]:
import pandas as pd
from datetime import datetime

# Assuming df['time'] is your initial DataFrame column with time strings

# Define the function to categorize time of day
def get_time_of_day(hour):
    if 0 <= hour < 5:           # 'Early Morning'
        return 0.0
    elif 5 <= hour < 12:        # 'Morning'
        return 1.0
    elif 12 <= hour < 17:       # 'Afternoon'
        return 2.0
    elif 17 <= hour < 21:       # 'Evening'
        return 3.0
    else:                       # 'Night'
        return 4.0

# Convert 'time' column to datetime objects, ignoring the timezone for simplicity
df['time_parsed'] = pd.to_datetime(df['time'].str.split('+').str[0], format='%H:%M:%S.%f')

# Extract hour from the 'time_parsed' column
df['hour'] = df['time_parsed'].dt.hour

# Apply the get_time_of_day function to create 'time_of_day' column
df['time_of_day'] = df['hour'].apply(get_time_of_day)

# Now df includes the new 'time_of_day' column
print(df[['time', 'time_of_day']])


                   time  time_of_day
0    01:00:50.174233+00          0.0
1    01:02:51.384714+00          0.0
2    01:04:52.127056+00          0.0
3    01:06:50.205974+00          0.0
4    01:08:50.162332+00          0.0
..                  ...          ...
109  04:50:16.123489+00          0.0
110  04:52:18.185763+00          0.0
111  04:54:16.141528+00          0.0
112  04:56:20.078654+00          0.0
113  04:58:16.116687+00          0.0

[1340 rows x 2 columns]


In [7]:
df

,index,time,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,13WindDirection,...,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1,time_parsed,hour,time_of_day
0,01_1,01:00:50.174233+00,97.3,21.8,0.0,9.57,0.0,0.0,0.0,6.0,...,0.11,8.3,21.0,95.0,87.0,23.0,0.0,1900-01-01 01:00:50.174233,1,0.0
1,01_2,01:02:51.384714+00,96.6,22.0,0.0,9.81,0.0,0.0,0.0,6.0,...,0.11,8.3,21.0,95.0,87.0,22.0,0.0,1900-01-01 01:02:51.384714,1,0.0
2,01_3,01:04:52.127056+00,95.9,22.2,0.0,10.22,0.0,0.0,0.0,5.0,...,0.11,8.3,21.0,95.0,87.0,0.0,0.0,1900-01-01 01:04:52.127056,1,0.0
3,01_4,01:06:50.205974+00,95.3,22.4,0.0,10.25,0.0,0.0,0.0,6.0,...,0.11,8.3,21.0,95.0,87.0,22.0,0.0,1900-01-01 01:06:50.205974,1,0.0
4,01_5,01:08:50.162332+00,93.8,22.4,0.0,10.45,0.0,0.0,0.0,6.0,...,0.11,8.3,21.0,95.0,87.0,23.0,0.0,1900-01-01 01:08:50.162332,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,12_110,04:50:16.123489+00,43.0,33.0,0.0,52.30,0.0,0.0,1.0,5.0,...,0.11,5.0,21.0,95.0,87.0,109.0,0.0,1900-01-01 04:50:16.123489,4,0.0
110,12_111,04:52:18.185763+00,42.5,32.5,0.0,53.76,0.0,0.0,0.8,4.0,...,0.11,5.0,21.0,95.0,87.0,0.0,0.0,1900-01-01 04:52:18.185763,4,0.0
111,12_112,04:54:16.141528+00,41.7,33.1,0.0,54.86,0.0,0.0,0.8,4.0,...,0.11,5.1,21.0,95.0,87.0,109.0,0.0,1900-01-01 04:54:16.141528,4,0.0
112,12_113,04:56:20.078654+00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,2.0,...,0.11,5.0,21.0,95.0,87.0,104.0,0.0,1900-01-01 04:56:20.078654,4,0.0


In [8]:
df.describe(include='all')

,index,time,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,13WindDirection,...,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1,time_parsed,hour,time_of_day
count,1340,1340,1340.000000,1340.000000,1340.000000,1340.000000,1340.0,1340.0,1340.000000,1340.000000,...,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340,1340.000000,1340.000000
unique,1340,1340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,01_1,01:00:50.174233+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,48.565149,28.975373,21.761828,27.479515,0.0,0.0,0.751045,2.914179,...,0.118567,6.512910,20.721642,89.968657,83.330597,40.767164,0.403731,1900-01-01 04:27:40.767920640,3.958955,0.329104
min,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1900-01-01 00:26:03.264438,0.000000,0.000000
25%,NaN,NaN,41.800000,25.900000,0.000000,5.475000,0.0,0.0,0.100000,1.000000,...,0.110000,5.000000,21.000000,94.000000,87.000000,23.000000,0.000000,1900-01-01 02:30:48.188879104,2.000000,0.000000
50%,NaN,NaN,45.700000,29.600000,0.000000,11.005000,0.0,0.0,0.600000,2.000000,...,0.110000,7.000000,22.000000,97.000000,89.000000,29.000000,0.000000,1900-01-01 04:00:46.233681408,4.000000,0.000000
75%,NaN,NaN,52.725000,32.300000,65.530000,55.992500,0.0,0.0,1.100000,4.000000,...,0.120000,7.400000,23.000000,100.000000,93.000000,52.000000,1.000000,1900-01-01 05:37:00.989231360,5.000000,1.000000
max,NaN,NaN,97.300000,38.400000,65.530000,65.520000,0.0,0.0,5.000000,7.000000,...,0.210000,8.300000,33.000000,123.000000,116.000000,123.000000,1.000000,1900-01-01 13:03:34.487929,13.000000,2.000000


In [9]:
list(df)

['index',
 'time',
 '1Air Humidity',
 '1AirTemperature',
 '1LightHigh',
 '1LightLow',
 '11Rain Accumulate (mm)',
 '11Rain count',
 '12WindSpeed',
 '13WindDirection',
 '23SoilWC ค.ชื้นดิน',
 '23SoilTemp.อุณหภูมิดิน',
 '23SoilEC',
 '23SoilpH',
 '26N',
 '26P',
 '26K',
 '29SoilTension',
 'flow1',
 'time_parsed',
 'hour',
 'time_of_day']

In [10]:
df = df.drop(['index','time','11Rain Accumulate (mm)','11Rain count','time_parsed',],axis=1)

In [11]:
df.describe(include='all')

,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,12WindSpeed,13WindDirection,23SoilWC ค.ชื้นดิน,23SoilTemp.อุณหภูมิดิน,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1,hour,time_of_day
count,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000
mean,48.565149,28.975373,21.761828,27.479515,0.751045,2.914179,34.977164,21.970224,0.118567,6.512910,20.721642,89.968657,83.330597,40.767164,0.403731,3.958955,0.329104
std,11.594615,4.606203,30.873723,24.924393,0.746727,2.202777,19.713135,1.738520,0.019267,1.240398,7.396645,30.364609,28.248795,31.291224,0.490828,2.702686,0.518417
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,41.800000,25.900000,0.000000,5.475000,0.100000,1.000000,31.400000,21.500000,0.110000,5.000000,21.000000,94.000000,87.000000,23.000000,0.000000,2.000000,0.000000
50%,45.700000,29.600000,0.000000,11.005000,0.600000,2.000000,36.900000,22.000000,0.110000,7.000000,22.000000,97.000000,89.000000,29.000000,0.000000,4.000000,0.000000
75%,52.725000,32.300000,65.530000,55.992500,1.100000,4.000000,42.500000,23.100000,0.120000,7.400000,23.000000,100.000000,93.000000,52.000000,1.000000,5.000000,1.000000
max,97.300000,38.400000,65.530000,65.520000,5.000000,7.000000,80.200000,23.700000,0.210000,8.300000,33.000000,123.000000,116.000000,123.000000,1.000000,13.000000,2.000000


In [17]:
print(set(df['time_of_day']))

{0.0, 1.0, 2.0}


<hr>

<h4>03 Exploratory Data Analysis (EDA)</h4>

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
# # Pair Plot
# sns.pairplot(df)
# plt.show()

In [ ]:
def remove_outliers(df):
    cleaned_data = df.copy()
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            cleaned_data = cleaned_data[(cleaned_data[col] >= lower_bound) & (cleaned_data[col] <= upper_bound)]
    return cleaned_data

In [ ]:
# Remove outliers from the dataset
df = remove_outliers(df)
df

In [ ]:
df.describe().T

In [ ]:
# # Pair Plot
# sns.pairplot(df)
# plt.show()

<h4>04 Feature Engineering</h4>

In [ ]:
df.columns

In [ ]:
df

<h5>Feature Importance</h5>

In [ ]:
dfx = df
list(dfx)

In [ ]:
# dfx = dfx.drop(['index','time','11Rain Accumulate (mm)','11Rain count'],axis=1)
dfx = df

<h5>relationship</h5>

In [ ]:
dfx.corr().style.format("{:.4}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

In [ ]:
x = {'1Air Humidity':-0.1601,
'1AirTemperature':0.3111,
'1LightHigh':0.06379,
'1LightLow':0.1204,
'12WindSpeed':-0.1083,
'13WindDirection':0.08821,
'23SoilWC ค.ชื้นดิน':1,
'23SoilTemp.อุณหภูมิดิน':0.2677,
'23SoilEC':0.4955,
'23SoilpH':0.1313,
'26N':0.3578,
'26P':0.4464,
'26K':0.426,
'29SoilTension':-0.1967,
'flow1':-0.2115,
}
# Sort the dictionary by its values
sorted_x = dict(sorted(x.items(), key=lambda item: item[-1],reverse=True))

sorted_x

In [ ]:
# X = dfx[['1Air Humidity', '1AirTemperature', '1LightHigh',
#                 '1LightLow','12WindSpeed',
#                 '23SoilEC', '23SoilpH', '23SoilTemp.อุณหภูมิดิน',  '26N', '26P', '26K',
#                 '29SoilTension', 'flow1']]

X = dfx[['26P', '26K', '26N', '1AirTemperature', '23SoilTemp.อุณหภูมิดิน','23SoilpH', '1LightLow']]     
# X = dfx[['23SoilEC', '26P', '26K', '1AirTemperature', '23SoilpH', '1LightLow']]       #  'XGBoost': 1.1382227172851562,    
# X = dfx[['23SoilEC','26P','26K','26N','1AirTemperature','23SoilTemp.อุณหภูมิดิน','23SoilpH','1LightLow']]  #'XGBoost': 1.1545134582519532,

y = df[['23SoilWC ค.ชื้นดิน']]

In [ ]:
X

In [ ]:
y

<hr>

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.078, random_state = 99)
X_train.shape, X_test.shape

<h4>Scaling?</h4>

In [ ]:
from sklearn.preprocessing import RobustScaler

In [ ]:
cols = X_train.columns

In [ ]:
X_train

In [ ]:
# scaler = RobustScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [ ]:
# X_train = pd.DataFrame(X_train, columns=[cols])
# X_test = pd.DataFrame(X_test, columns=[cols])

In [ ]:
# 10/0

<hr>

<h4>05 Model Selection</h4>

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import BayesianRidge
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error


from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from pygam import LinearGAM, s, f
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler


In [ ]:
# 07 Decision Tree Regression
dt_param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 10, 20]
}
dt_grid_search = GridSearchCV(DecisionTreeRegressor(), dt_param_grid, cv=5, scoring='neg_mean_absolute_error')
dt_grid_search.fit(X_train, y_train)
dt_best = dt_grid_search.best_estimator_
dt_predictions = dt_best.predict(X_test)
dt_mae = mean_absolute_error(y_test, dt_predictions)
print(f'Decision Tree MSE: {dt_mae}, Best Params: {dt_grid_search.best_params_}')

#####################################################################################################################################
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import mean_absolute_error

# # Expanded hyperparameter grid
# dt_param_grid = {
#     'max_depth': [None, 10, 20, 30, 40],
#     'min_samples_split': [2, 10, 20, 30],
#     'min_samples_leaf': [1, 2, 4, 6],  # Minimum number of samples required at each leaf node
#     'max_features': ['auto', 'sqrt', 'log2', None],  # Number of features to consider when looking for the best split
#     'ccp_alpha': [0.0, 0.01, 0.1]  # Complexity parameter used for Minimal Cost-Complexity Pruning
# }

# # Grid Search with Cross-Validation
# dt_grid_search = GridSearchCV(DecisionTreeRegressor(random_state=42), dt_param_grid, cv=5, scoring='neg_mean_absolute_error')
# dt_grid_search.fit(X_train, y_train)

# # Best model and predictions
# dt_best = dt_grid_search.best_estimator_
# dt_predictions = dt_best.predict(X_test)

# # Evaluate performance
# dt_mae = mean_absolute_error(y_test, dt_predictions)
# print(f'Decision Tree MAE: {dt_mae}, Best Params: {dt_grid_search.best_params_}')

In [ ]:
# 08 Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()
rfr_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10]
}
rfr_grid_search = GridSearchCV(rfr, rfr_param_grid, cv=5, scoring='neg_mean_absolute_error')
rfr_grid_search.fit(X_train, y_train)
rfr_best = rfr_grid_search.best_estimator_
rfr_predictions = rfr_best.predict(X_test)
rfr_mae = mean_absolute_error(y_test, rfr_predictions)
print("Random Forest Regressor mae:", rfr_mae)

#####################################################################################################################################
# # Enhanced hyperparameter grid
# rfr_param_grid = {
#     'n_estimators': [100, 200, 300],  # More options for the number of trees
#     'max_depth': [None, 5, 10, 20],  # Additional depth options for tree growth
#     'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
#     'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
#     'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider when looking for the best split
# }

# # Grid Search with Cross-Validation
# rfr_grid_search = GridSearchCV(RandomForestRegressor(random_state=42), rfr_param_grid, cv=5, scoring='neg_mean_absolute_error')
# rfr_grid_search.fit(X_train, y_train)

# # Best model and predictions
# rfr_best = rfr_grid_search.best_estimator_
# rfr_predictions = rfr_best.predict(X_test)

# # Evaluate performance
# rfr_mae = mean_absolute_error(y_test, rfr_predictions)
# print(f'Random Forest Regressor MAE: {rfr_mae}, Best Params: {rfr_grid_search.best_params_}')


In [ ]:
# 09 Gradient Boosting Regression (XGBoost)
import xgboost as xgb

xgb_param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 6, 9]
}
xgb_grid_search = GridSearchCV(xgb.XGBRegressor(), xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
xgb_grid_search.fit(X_train, y_train)
xgb_best = xgb_grid_search.best_estimator_
xgb_predictions = xgb_best.predict(X_test)
xgb_mae = mean_absolute_error(y_test, xgb_predictions)
print(f'XGBoost MSE: {xgb_mae}, Best Params: {xgb_grid_search.best_params_}')
#####################################################################################################################################
# # Extended hyperparameter grid
# xgb_param_grid = {
#     'n_estimators': [100, 200, 300],
#     'learning_rate': [0.01, 0.05, 0.1, 0.2],
#     'max_depth': [3, 6, 9, 12],
#     'subsample': [0.6, 0.8, 1.0],  # Fraction of observations to be randomly sampled for each tree.
#     'colsample_bytree': [0.6, 0.8, 1.0],  # Fraction of features to be randomly sampled for each tree.
#     'gamma': [0, 0.1, 0.2]  # Minimum loss reduction required to make a further partition on a leaf node.
# }

# # Grid Search with Cross-Validation
# xgb_grid_search = GridSearchCV(xgb.XGBRegressor(objective='reg:squarederror', random_state=42), xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
# xgb_grid_search.fit(X_train, y_train)

# # Best model and predictions
# xgb_best = xgb_grid_search.best_estimator_
# xgb_predictions = xgb_best.predict(X_test)

# # Evaluate performance
# xgb_mae = mean_absolute_error(y_test, xgb_predictions)
# print(f'XGBoost MAE: {xgb_mae}, Best Params: {xgb_grid_search.best_params_}')


In [ ]:
# Gradient Boosting Regression Trees
gbrt = GradientBoostingRegressor()
gbrt_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.05, 0.1, 0.2]
    }
gbrt_grid_search = GridSearchCV(gbrt, gbrt_param_grid, cv=5, scoring='neg_mean_absolute_error')
gbrt_grid_search.fit(X_train, y_train)
gbrt_best = gbrt_grid_search.best_estimator_
gbrt_predictions = gbrt_best.predict(X_test)
gbrt_mae = mean_absolute_error(y_test, gbrt_predictions)
print(f"{gbrt_mae} Gradient Boosting Regression")


In [ ]:
# 10 Gradient Boosting Regression (LightGBM)
import lightgbm as lgb

lgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth':[3, 6, 9, -1],
    'num_leaves': [31, 62, 127, 255]
}
lgb_grid_search = GridSearchCV(lgb.LGBMRegressor(), lgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
lgb_grid_search.fit(X_train, y_train)
lgb_best = lgb_grid_search.best_estimator_
lgb_predictions = lgb_best.predict(X_test)
lgb_mae = mean_absolute_error(y_test, lgb_predictions)

print(f"{lgb_mae} Gradient Boosting Regression")
# #####################################################################################################################################



In [ ]:
# K-Nearest Neighbors Regression (KNN)

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

knn_param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}
knn_grid_search = GridSearchCV(KNeighborsRegressor(), knn_param_grid, cv=5, scoring='neg_mean_absolute_error')
knn_grid_search.fit(X_train, y_train)
knn_best = knn_grid_search.best_estimator_
knn_predictions = knn_best.predict(X_test)
knn_mae = mean_absolute_error(y_test, knn_predictions)
print(f'KNN Regression MAE: {knn_mae}, Best Params: {knn_grid_search.best_params_}')

#####################################################################################################################################
# # Scale the features
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Optionally apply PCA for dimensionality reduction
# # pca = PCA(n_components='mle')  # mle lets PCA decide the number of components to keep
# # X_train_scaled = pca.fit_transform(X_train_scaled)
# # X_test_scaled = pca.transform(X_test_scaled)

# # Expanded hyperparameter grid
# knn_param_grid = {
#     'n_neighbors': range(1, 10),  # Consider testing a broader range of neighbors
#     'weights': ['uniform', 'distance'],
#     'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
#     # 'metric': ['euclidean', 'manhattan', 'minkowski'],  # Optional: Explore different distance metrics
# }

# # Grid Search with Cross-Validation
# knn_grid_search = GridSearchCV(KNeighborsRegressor(), knn_param_grid, cv=5, scoring='neg_mean_absolute_error')
# knn_grid_search.fit(X_train_scaled, y_train)

# # Best model and predictions
# knn_best = knn_grid_search.best_estimator_
# knn_predictions = knn_best.predict(X_test_scaled)

# # Evaluate performance
# knn_mae = mean_absolute_error(y_test, knn_predictions)
# print(f'KNN Regression MAE: {knn_mae}, Best Params: {knn_grid_search.best_params_}')


In [ ]:
# # # 12 Neural Network Regression
# from sklearn.neural_network import MLPRegressor
# from sklearn.preprocessing import StandardScaler

# nn_param_grid = {
#     'hidden_layer_sizes': [(50,), (100,), (50,50)],
#     'activation': ['relu', 'tanh', 'logistic'],
#     'solver': ['adam', 'lbfgs']
# }
# scaler = StandardScaler().fit(X_train)
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# nn_grid_search = GridSearchCV(MLPRegressor(max_iter=1000), nn_param_grid, cv=5, scoring='neg_mean_absolute_error')
# nn_grid_search.fit(X_train_scaled, y_train)
# nn_best = nn_grid_search.best_estimator_
# nn_predictions = nn_best.predict(X_test_scaled)
# nn_mae = mean_absolute_error(y_test, nn_predictions)
# print(f'Neural Network Regression MAE: {nn_mae}, Best Params: {nn_grid_search.best_params_}')

# #####################################################################################################################################
# # # Expanded hyperparameter grid
# # nn_param_grid = {
# #     'hidden_layer_sizes': [(50,), (100,), (50,50), (100, 100), (50, 100, 50)],
# #     'activation': ['relu', 'tanh', 'logistic'],
# #     'solver': ['adam', 'lbfgs'],
# #     'alpha': [0.0001, 0.001, 0.01],  # L2 penalty (regularization term) parameter.
# #     'learning_rate_init': [0.001, 0.01, 0.1],  # The initial learning rate used.
# # }

# # # Scale features
# # scaler = StandardScaler().fit(X_train)
# # X_train_scaled = scaler.transform(X_train)
# # X_test_scaled = scaler.transform(X_test)

# # # Early stopping setup
# # early_stopping = {'n_iter_no_change': 10, 'validation_fraction': 0.1, 'tol': 0.001}

# # # Grid Search with Cross-Validation
# # nn_grid_search = GridSearchCV(MLPRegressor(max_iter=1000, **early_stopping), nn_param_grid, cv=5, scoring='neg_mean_absolute_error')
# # nn_grid_search.fit(X_train_scaled, y_train)

# # # Best model and predictions
# # nn_best = nn_grid_search.best_estimator_
# # nn_predictions = nn_best.predict(X_test_scaled)

# # # Evaluate performance
# # nn_mae = mean_absolute_error(y_test, nn_predictions)
# # print(f'Neural Network Regression MAE: {nn_mae}, Best Params: {nn_grid_search.best_params_}')


In [ ]:
# Random Forest baseline model
rf = RandomForestRegressor()
rf.fit(X_train,y_train)
rf_baseline = rf.predict(X_test)
rfbl_mae = mean_absolute_error(y_test, rf_baseline)
print(f'RFR MAE: {rfbl_mae}')

In [ ]:
# # XGB baseline model
# xb = xgb.XGBRegressor()
# xb.fit(X_train, y_train)
# y_hat = xb.predict(X_test)
# xbbl_mae = mean_absolute_error(y_test, y_hat)
# print(f'XGB MAE: {xbbl_mae}')

# #####################################################################################################################################

param_grid = {
        "colsample_bytree": [0.3, 0.4, 0.5],
        "gamma": [0],
        "learning_rate": [0.4, 0.5, 0.6],
        "max_depth": [1, 2, 3],
        "reg_lambda": [11, 12, 13],
        "scale_pos_weight": [0],
        "subsample": [0.4, 0.5, 0.6],
}
grid_cv = xgb.XGBRegressor()
grid_cv = GridSearchCV(
    grid_cv,
    param_grid,
    n_jobs=-1,
    cv=3,
    scoring='neg_mean_absolute_error'
)
grid_cv.fit(X_train, y_train)
grid_cv.best_params_
y_hat = grid_cv.predict(X_test)
xbbl_mae = mean_absolute_error(y_test, y_hat)
print(f'XGB MAE: {xbbl_mae}')

In [ ]:
print(f'{dt_mae} Decision Tree MSE: , Best Params: {dt_grid_search.best_params_}')
print(f"{rfr_mae} Random Forest Regressor mae:", )
print(f'{xgb_mae} XGBoost MSE: , Best Params: {xgb_grid_search.best_params_}')
print(f"{gbrt_mae} Gradient Boosting Regression2")
print(f"{lgb_mae} Gradient Boosting Regression2")
print(f'{knn_mae} Best KNN Model MAE:  Best Parameters: {knn_grid_search.best_params_}')
print(f'{xbbl_mae} XGBoost baseline')
print(f"{rfbl_mae} Random Forest Regressor mae baseline")

In [ ]:
feature_bank = {
"Decision Tree":dt_mae,
"Random Forest Regressor":rfr_mae,
"XGBoost":xgb_mae,
"Gradient Boosting Regression":gbrt_mae,
"Gradient Boosting Regression2":lgb_mae,
"Best KNN Model":knn_mae,
# "Neural Network Regression":nn_mae,
"XGBoost baseline":xbbl_mae,
"Random Forest Regressor mae baseline":rfbl_mae
}

In [ ]:
feature_bank

In [ ]:
# Convert the dictionary to a DataFrame
dff = pd.DataFrame(list(feature_bank.items()), columns=['Model', 'MAE'])

# Sort the DataFrame by MAE
df_sorted = dff.sort_values('MAE')

# Create the bar plot
ax = sns.barplot(x='MAE', y='Model', data=df_sorted)

plt.title('Model Performance Comparison (Lower is Better)')
plt.xlabel('Mean Absolute Error (MAE)')
plt.ylabel('Model')

# Loop over the bars and use plt.text() to display the MAE value on each bar
for p in ax.patches:
    width = p.get_width()
    plt.text(width + 0.02,  # x-coordinate position of text
             p.get_y() + p.get_height() / 2,  # y-coordinate position of text
             '{:1.5f}'.format(width),  # Text to be displayed
             va='center')  # Center alignment of text

plt.grid(alpha=0.3)
plt.show()

<hr>

<hr>

<h4>06 Model Training</h4>

In [ ]:
x = []
src2 = r"C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\test\test"
for file in os.listdir(src2):
    print(os.path.join(src2,file))
    x.append(pd.read_csv(os.path.join(src2,file)))
x = pd.concat(x)
x = x.fillna(0)
x

In [ ]:
X.columns

In [ ]:
x = x[X.columns]
x

In [ ]:
# scaler = RobustScaler()
# x_test = scaler.fit_transform(x_test)
# x_test = scaler.transform(x_test)
# x_test = pd.DataFrame(x_test, columns=[cols])

In [ ]:
y = xgb_best.predict(x)
y

In [ ]:
len(y)

<hr>

<h4>07 Create SSV</h4>

In [ ]:
submission = pd.read_csv(r"C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\sample_submission.csv")
submission

In [ ]:
submission['23SoilWC ค.ชื้นดิน'].iloc[1:] = y[1:]
submission

In [ ]:
10/0
submission.to_csv('v.0.6.0.csv',index=False)